 # Basic Candlestick Pattern Detection
 
 This notebook demonstrates the basic candlestick pattern detection functionalities from the 
 `patternforge.candlestick.patterns` module. We'll cover:
 
 1. Setting up the environment
 2. Loading sample data
 3. Basic single-candle patterns
 4. Two-candle patterns
 5. Three-candle patterns
 6. Visualizing detected patterns


## 1. Setting up the environment

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import yfinance as yf
from datetime import datetime, timedelta
import sys
import os

# Add the parent directory to the path so we can import the patternforge module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from patternforge.candlestick.patterns import CandlestickPatterns